In [1]:
# Import libraries
from functions import *

import json
import os
import subprocess
import time

# Creating a P2PKH transaction

In this section we'll create a P2PKH transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and we'll test it using bitcoin core in regtest mode.

## Reading
- Andreas Antonopoulos - Mastering Bitcoin Chapter 6
- Jimmy Song - Programming Bitcoin Chapters 5 and 7 

## Setup 

### Requirements
For this exercise we'll need Bitcoin Core (v22 or higher) with the application data is stored in 
```$HOME/Library/Application Support/Bitcoin```.

### Setup bitcoind in regtest
Start up regtest mode, delete any regtest network history so we are starting from scratch. Create a wallet so that we can fund our first output using the `sendtoaddress` command. Mine 101 blocks so that the mining reward first block will have sufficient block maturity (100 blocks) to spend from.

In [2]:
# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(2) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# Start up bitcoind in regtest mode
os.system("bitcoind -regtest -daemon -fallbackfee=0.0002")
time.sleep(1.5)

# Create a new wallet and address that we can mine blocks so that we can fund our transactions
wallet = subprocess.getoutput("bitcoin-cli -regtest createwallet mywallet")
address = subprocess.getoutput("bitcoin-cli -regtest getnewaddress")

# Generate 101 blocks so that the first block's block reward reaches maturity
result = subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 101 {addr}".format(addr=address))

# Check that we were able to mine 101 blocks
blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")
assert(blockcount == "101")

### Create a P2PKH UTXO

In order to create a transaction spending from a P2PKH UTXO, we'll first need to create the UTXO that is locked with a p2pkh script. To do that, we'll create a P2PKH address from a private key, and fund it using the bitcoind wallet created in the setup step.

#### Create a p2pkh address 
For more on this step, review the 'Addresses' notebook.

In [3]:
sender_privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
sender_pubkey = privkey_to_pubkey(sender_privkey)
sender_p2pkh_addr = pk_to_p2pkh(sender_pubkey, network = "regtest")
print("sender's p2pkh address: " + sender_p2pkh_addr)

sender's p2pkh address: n4XmX91N5FfccY678vaG1ELNtXh6skVES7


#### Fund the 'sender' with 2.001 btc (0.001 btc is for the next tx fee)

In [4]:
txid_to_spend = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + sender_p2pkh_addr + " 2.001")
print(txid_to_spend)

33a4713b61f9472c874b0ac7c4c5e598065568dab983fbdd35fb75bb0c2e9982


We can view the transaction using the bitcoin-cli commands `getrawtransaction` and `decoderawtransaction` as follows:

In [5]:
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_to_spend)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
print(decoded)

{
  "txid": "33a4713b61f9472c874b0ac7c4c5e598065568dab983fbdd35fb75bb0c2e9982",
  "hash": "b97df669b77cd556f2cd651a8c5ce721a25d5089969c26c769372d843e1e06f7",
  "version": 2,
  "size": 228,
  "vsize": 147,
  "weight": 585,
  "locktime": 101,
  "vin": [
    {
      "txid": "36d28be5523744f943d36dccf971122cee4a79d0fdb945b6c19670d1047f76e8",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "304402202e72a0e478d17f1cfba41b46166f71b7ff87e7d42b9a0828d1842bfa3ed2ac0602203cffce896c60b291513e7d32730fae9540106ccb4eae9bfc68aa177f2835b72801",
        "02c51f10db16f522fe2ec15f3804dab50770affb69461d5c761cb5321d773cbb0a"
      ],
      "sequence": 4294967294
    }
  ],
  "vout": [
    {
      "value": 2.00100000,
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 fc7250a211deddc70ee5a2738de5f07817351cef OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(n4XmX91N5FfccY678vaG1ELNtXh6skVES7)#n2xufj4n",
        "hex": "7

#### Find which output index the btc was sent to
Since we only sent 2.001 btc of the coinbase transaction (50 btc) to our address, bitcoind creates a change output to send the rest of the btc. By looking at the outputs we can see which is the change output and which was sent to our address. To do this in python we can do the following:

In [6]:
d = json.loads(decoded)
if d["vout"][0]["scriptPubKey"]["address"] == sender_p2pkh_addr:
    index_to_spend = 0
elif d["vout"][1]["scriptPubKey"]["address"] == sender_p2pkh_addr:
    index_to_spend = 1
else:
    raise Exception("couldn't find output")
print("index to spend from: " + str(index_to_spend))

index to spend from: 0


#### Mine a block so that the funding tx gets confirmed

In [7]:
subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 1 {addr}".format(addr=address));

## Spending a p2pkh UTXO

Now that we have some funds locked up in a p2pkh utxo, we can create a transaction spending from it. Let's say we want to send 1.5 btc to the address `mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE`.

### Decoding a base58 address

The first thing we need to do is decode the address by doing this we can achieve the following:
1 - validate the checksum to know the address was transmitted without error
2 - make sure we are sending btc on the correct network (testnet/mainnet)
3 - know what to put in the scriptPubkey

For more on addresses, refer back to the 'Addresses' chapter.

In [8]:
receiver_address = 'mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE'
receiver_address_decoded = decode_base58(receiver_address)
# Darius - TODO: create a function in the address chapter to validate and parse addresses and use here

prefix = receiver_address_decoded[0]  
pubkey_hash = receiver_address_decoded[1:-4] 
checksum = receiver_address_decoded[-4:]
print(hex(prefix))
print(pubkey_hash.hex())
print(checksum.hex())

0x6f
3bc28d6d92d9073fb5e3adf481795eaf446bceed
ee2161b7


The first byte , in our case `6f`, tells us that this address corresponds to a p2pkh output for testnet. For more on decoding addresses, refer back to the 'Addresses' chapter.

Now we can create the receiver's output scriptPubkey:

In [9]:
receiver_spk = bytes.fromhex("76a914") + pubkey_hash + bytes.fromhex("88ac")

### Create an unsigned p2pkh transaction

The first thing we'll do is define the inputs and outputs of our transaction.

In [10]:
# Note we have already defined a few variables we need to create our transaction:
# The input utxo txid and index: `txid_to_spend` and `index_to_spend`
# The input private key and public key: `sender_privkey` and `sender_pubkey`

# Set our outputs
# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
change_pubkey = privkey_to_pubkey(change_privkey)

# Determine our output scriptPubkeys and amounts (in satoshis)
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("76a914") + hash160(change_pubkey) + bytes.fromhex("88ac")

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the input's scriptSig, the field where the signature goes, is empty. This first step is necessary as the signature will cover the whole transaction (using SIGHASH_ALL). In a later chapter we will cover other sighash types and how they are signed.

In [11]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  020000000182992e0cbb75fb35ddfb83b9da68550698e5c5c4c70a4b872c47f9613b71a4330000000000ffffffff0280d1f008000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac80f0fa02000000001976a914531260aa2a199e228c537dfa42c82bea2c7c1f4d88ac00000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the scriptSig.

In [12]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + unsigned_tx.hex())
print(decoded)

{
  "txid": "e71b12d8b544503c610db36f3eabfec2372e030699acf1574105a50bc3aa8d66",
  "hash": "e71b12d8b544503c610db36f3eabfec2372e030699acf1574105a50bc3aa8d66",
  "version": 2,
  "size": 119,
  "vsize": 119,
  "weight": 476,
  "locktime": 0,
  "vin": [
    {
      "txid": "33a4713b61f9472c874b0ac7c4c5e598065568dab983fbdd35fb75bb0c2e9982",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 1.50000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 3bc28d6d92d9073fb5e3adf481795eaf446bceed OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE)#xlnzfr97",
        "hex": "76a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac",
        "address": "mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE",
        "type": "pubkeyhash"
      }
    },
    {
      "value": 0.50000000,
      "n": 1,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 531260a

Before we can sign this transaction there is one final step we need to do. We need to replace the empty scriptSig with the scriptPubkey of the input we are signing over. If we had multiple inputs, we would need to do this step for each input. We will cover signing transactions with multiple inputs in a later chapter.

Since we are spending from a p2pkh utxo, we will create the scriptPubkey in the same way as we did for the outputs, but using the sender's pubkey:

In [13]:
pk_hash = hash160(sender_pubkey)
input_spk = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

# replace the empty scriptSig with the input scriptPubkey
inputs = (
    txid
    + index
    + varint_len(input_spk)
    + input_spk
    + sequence
)

# tx hex to sign
tx_to_sign = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

Now we are ready to hash this transaction and produce an ecdsa signature on it. 

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [14]:
# Append the sighash flag to the transaction
sighash_flag = bytes.fromhex("0100 0000") # SIGHASH_ALL
sighash_preimage = tx_to_sign + sighash_flag

# Create sigHash to be signed
sighash = hash256(sighash_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(sender_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Signature
sig_script_signed = (
    pushbytes(signature)
    + pushbytes(sender_pubkey)
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(sig_script_signed)
    + sig_script_signed
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  020000000182992e0cbb75fb35ddfb83b9da68550698e5c5c4c70a4b872c47f9613b71a433000000006a47304402204e24c39774d935b9b5a3b79a27de830247af44ceb5f3fddc5f0e5b6c4bed7b7a02206df7a1fbd9811c871ca1fa9eea4831aa656e207ebe3f6aa77be444a9a9ee81470121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aaffffffff0280d1f008000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac80f0fa02000000001976a914531260aa2a199e228c537dfa42c82bea2c7c1f4d88ac00000000


### Broadcast the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [15]:
new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + signed_tx.hex())
# new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest testmempoolaccept " + "'[\"" +  signed_tx.hex()+ "\"]'")

print(new_tx_txid)

5f325bfa44afca63798fbe341b9c82f78187afcff59fa501177a7045791a31b3


We can decode the serialized transaction using ```decoderawtransction```. Notice that our output addresses match the change and receiver addresses from earlier.

In [16]:
print("receiver's p2pkh address: " + receiver_address)
change_p2pkh_addr = pk_to_p2pkh(change_pubkey, network = "regtest")
print("sender's change p2pkh address: " + change_p2pkh_addr)

receiver's p2pkh address: mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE
sender's change p2pkh address: mo6CPsdW8EsnWdmSSCrQ6225VVDtpMBTug


In [17]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + signed_tx.hex())
print(decoded)

{
  "txid": "5f325bfa44afca63798fbe341b9c82f78187afcff59fa501177a7045791a31b3",
  "hash": "5f325bfa44afca63798fbe341b9c82f78187afcff59fa501177a7045791a31b3",
  "version": 2,
  "size": 225,
  "vsize": 225,
  "weight": 900,
  "locktime": 0,
  "vin": [
    {
      "txid": "33a4713b61f9472c874b0ac7c4c5e598065568dab983fbdd35fb75bb0c2e9982",
      "vout": 0,
      "scriptSig": {
        "asm": "304402204e24c39774d935b9b5a3b79a27de830247af44ceb5f3fddc5f0e5b6c4bed7b7a02206df7a1fbd9811c871ca1fa9eea4831aa656e207ebe3f6aa77be444a9a9ee8147[ALL] 034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa",
        "hex": "47304402204e24c39774d935b9b5a3b79a27de830247af44ceb5f3fddc5f0e5b6c4bed7b7a02206df7a1fbd9811c871ca1fa9eea4831aa656e207ebe3f6aa77be444a9a9ee81470121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa"
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 1.50000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH1

## Quiz
- 1. When we generated our addresses using the function `pk_to_p2pkh`, we passed in an argument `"regtest"` for the network. If instead we had passed in `"mainnet"` to `pk_to_p2pkh`, what would have happened in the next step where we fund the address using `bitcoin-cli -regtest sendtoaddress`?
    - A. Nothing different would happen as the address still encodes the same pubkey hash.
    - B. `sendtoaddress` would work but with a warning that a mainnet address is being used on regtest.
    - C. `sendtoaddress` would return an error saying that the address is invalid. 
    - D. `sendtoaddress` would work but later on when we try to spend the output, our signature would be invalid.

 ## Answers
    
- 1. C : Since we pass in the `-regtest` flag when running `sendtoaddress`, bitcoind it would say the address is invalid and not send bitcoin to it. Note that there's nothing stopping a bad wallet implementation from decoding the address and ignoring the network prefix. The purpose of this prefix is to prevent users accidentally sending mainnet bitcoins instead of testnet bitcoins. 

## Exercise
Now that we have the change output with a known private key `change_privkey`, create a transaction that spends from it to two outputs. The first output should to send 0.1 btc to the address `mgiS1dSDrPunE7GvXmoS4xEfmdwWsStZc7`. The second address should send the rest to a change output with the address `mz8AXDhDMhvLs7kxwfQxvcH5GoVH6AdARZ`. Set the miner fee to 0.001 btc.

The first step will be to decode the output addresses we want to send btc to and convert them into their scriptPubkeys.

## Solution

In [18]:
# Convert the addresses to scriptPubkeys
# TODO: create a function in the address chapter to validate an address and convert to a scriptPubkey
# Use that function here
receiver_address = 'mgiS1dSDrPunE7GvXmoS4xEfmdwWsStZc7'
receiver_address_decoded = decode_base58(receiver_address)
receiver_pubkey_hash = receiver_address_decoded[1:-4] 
output1_spk = bytes.fromhex("76a914") + receiver_pubkey_hash + bytes.fromhex("88ac")

change_address = 'mz8AXDhDMhvLs7kxwfQxvcH5GoVH6AdARZ'
change_address_decoded = decode_base58(change_address)
change_pubkey_hash = change_address_decoded[1:-4] 
output2_spk = bytes.fromhex("76a914") + change_pubkey_hash + bytes.fromhex("88ac")

Now let's create an unsigned transaction. For now we will keep the scriptSig empty (`0x00`)

In [19]:
version = bytes.fromhex("0200 0000")

# INPUTS
input_count = bytes.fromhex("01")

# INPUT 1
txid_to_spend = new_tx_txid
txid = (bytes.fromhex(txid_to_spend))[::-1]
index_to_spend = 1
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)
scriptsig = bytes.fromhex("00")
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value_sat = int(float("0.1") * 100000000)
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)

# OUTPUT 2
output2_value_sat = int(float("0.399") * 100000000) # 0.5 - 0.1 - 0.001 for the miner fee
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

# print(unsigned_tx.hex())

Replace the scriptSig with the input's scriptPubkey. You can either code this yourself or take it directly from the decoded transaction. The `"hex"` field of the output we are spending from can be used directly.

In [20]:
input_spk = bytes.fromhex("76a914531260aa2a199e228c537dfa42c82bea2c7c1f4d88ac")

In [21]:
inputs = (
    txid
    + index
    + varint_len(input_spk)
    + input_spk
    + sequence
)

tx_to_sign = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)


# Append the sighash flag to the transaction
sighash_flag = bytes.fromhex("0100 0000") # SIGHASH_ALL
sighash_preimage = tx_to_sign + sighash_flag

# Create sigHash to be signed
sighash = hash256(sighash_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(change_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Signature
sig_script_signed = (
    pushbytes(signature)
    + pushbytes(change_pubkey)
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(sig_script_signed)
    + sig_script_signed
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + locktime
)

# print("signed transaction: ",signed_tx.hex())

In [22]:
subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + signed_tx.hex())

'85b31f1ed7ec68838758398ac927c473ffca9770c4c83a73f1af457806eccc21'

In [23]:
# stop bitcoin core
subprocess.getoutput("bitcoin-cli -regtest stop")

'Bitcoin Core stopping'